# Docker Networking 

### Docker network
 
- ```bridge```: The default network driver. Bridge networks are isolated from the Docker host. You can create a bridge network with the ```docker network create <network-name>``` command.

- ```host```:  A host networks is used for standalone containers. It removes network isolation between the container and the Docker host to use the host’s networking configurations directly. That is, the container doesn't get its own IP address. It uses the host's address. Docker automatically creates a host network and you can't create more.

- ```overlay```: : Overlay networks connect multiple Docker daemons and enable swarm services to communicate with each other.  This strategy removes the need to do OS-level routing between containers. You can create an overlay network via a ```docker network create -d overlay --attachable <network-name>``` command. Note, that you need to initialize a swarm or join a Docker host to an existing swarm. You'll learn about the swarm later on.

> Besides these three network types, Docker also supports two others: ipvlan and macvlan. And if you do not want to use Docker networks, you can disable container networking completely.

You can find the network available to you using the docker network ls command.

```
$ docker network ls
NETWORK ID     NAME      DRIVER    SCOPE
7614b36c62f8   bridge    bridge    local
817d58bc7ecb   host      host      local
cf2953a94052   none      null      local
```

### Inspecting networks
Docker lets you inspect networks by using the ```docker network inspect <network-name>``` command. But what output does it give? To find the answer, create a network named hs-bridge and inspect it with the``` docker network inspect hs-bridge``` command.




The other property worth mentioning is Containers. It shows a list of containers attached to the given network. In this case, it doesn't represent any information since there are no containers attached to this network. If you had a container attached to this network, the output would include something similar to the snippet below.

<font color = 'yellow' size = 25>  Adding containers to a network


When you run a container without specifying the network, Docker automatically connects it to the default bridge network that you saw in the docker network ls output. If you want to specify another network you have several options.

The first approach is creating a container by specifying the network it will connect to when you run it. Assume you have an image named ubuntu:v1. You can use the following command to create a container without running it:

```
$ docker create --name hs-ubuntu-v1 --network hs-bridge ubuntu:v1

```

On the first line, you set the container name. Then the network you want the container to connect to while running. And finally, the image Docker should use to start the container. After running this command, check the container's network using the ```docker ps -a --format '{{ .ID }} {{ .Names }} {{ json .Networks }}'``` command. It lists all containers in the given format. Your result will look similar to the following snippet:

```
$ docker ps -a --format '{{ .ID }} {{ .Names }} {{ json .Networks }}'
0f3f713d745a ubuntu "bridge"
65fdf7f62a91 goofy_keller "bridge"
0417c27f8c28 sweet_bhaskara "bridge"
ee6381ad2782 hs-ubuntu-v1 "hs-bridge"
```

As you see, the hs-ubuntu container's network is hs-bridge. When you run this container it will run in the specified network.

The next approach is running a container specifying the desired network. To do that, use the --network <network-name> command when running the container setting like this:

```
$ docker run -d -it --name hs-ubuntu-v2 --network hs-bridge ubuntu:v1

```

> Now use the above mentioned ``` docker ps -a --format '{{ .ID }} {{ .Names }} {{ json .Networks }}' ``` command to check the created container network, it will be hs-bridge.

Another approach is to attach an already-running container to a network. This is another way to specify the network for containers. Use the following command on a running container:

```
$ docker network connect <network-name> <container-name>

```

Also, you can disconnect a container from the network by using the disconnect command instead of connect.

```
$ docker network disconnect <network-name> <container-name>


```

How to choose a network type?
You know about the different docker network types. Now, let's look at how you can choose which network type to use.

- bridge: This network is typically used when your applications run in standalone containers and you want them to communicate on the same host.

- host: This network offers better performance when making a large number of requests with different ports. It doesn't require NAT(Network address translation) which saves time.

- overlay: Such networks are useful when you want containers on different hosts to communicate with one another. Docker manages to route itself and provides secure communication between containers when encryption is enabled.